In [ ]:
from localization_dataset import KittiLlocalizationDataset
from localization import LocalizationNet, PoseRegressor
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim

from helpers import log

In [ ]:
DATA_PATH = None # TODO Change to config file data read

dataset = KittiLlocalizationDataset(DATA_PATH, "00")
loader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)

autoencoder_model = nn.DataParallel(LocalizationNet()).to('cuda')
autoencoder_model.train()
autoencoder_model.load_state_dict(torch.load("MapAutoEncoder.pth"))
trainable_params = sum(p.numel() for p in autoencoder_model.parameters() if p.requires_grad)
log("Trainable parameters:", trainable_params)

In [ ]:
def autoencoder_loss(true, pred):

    #rot, tr = true
    diff = (true-pred)**2
    loss = diff.mean()

    return loss*0.1

In [ ]:
def train(model, loader, optimizer, scheduler, log_vals=[]):
    for batch, blob in enumerate(loader):
        img1, rot_true, tr_true = blob

        img1 = torch.squeeze(img1).to('cuda')
        optimizer.zero_grad()
        transform_pred = model(img1)
        loss = autoencoder_loss(img1, transform_pred)
        
        log_vals.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        

        current = batch
        print("", end='\r')
        print("Iteration: %d / %d \t|\t Loss: %5f" % (current, len(loader), loss.item()), end='\n')



In [ ]:
import matplotlib.pyplot as plt
epochs = 2
log_vals = []
optimizer = optim.AdamW(autoencoder_model.parameters(), lr=1e-3, weight_decay=0.01)
#optimizer = optim.Adam(model.parameters(), lr=4*1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs*len(loader), eta_min=1e-5)

autoencoder_model.train()
print("============== ", "Training", " ==============\n")
for epoch in range(epochs):
    print("---------- ", "Epoch ", epoch, " ----------\n")
    train(autoencoder_model, loader, optimizer, scheduler, log_vals)
    plt.plot(log_vals)
    plt.show()

## Autoencoder test

In [ ]:
torch.save(autoencoder_model.state_dict(), 'MapAutoEncoder.pth')

In [ ]:
autoencoder_model.eval()
modul = autoencoder_model.module
img1, rot, tr = dataset[2000]
pred = modul(img1)
code = modul.get_code(img1)
img1 = torch.squeeze(img1).cpu().permute((1,  2, 0)).int().numpy()

plt.imshow(img1)
plt.show()
pred = torch.squeeze(pred).cpu().permute((1,  2, 0)).int().numpy()
plt.imshow(pred)
plt.show()
plt.imshow((img1-pred)**2)
plt.show()

In [ ]:
import numpy as np
img1, rot, tr = dataset[96]
img2, rot, tr = dataset[98]
plt.imsave("True0.png", torch.squeeze(img1).cpu().permute((1,  2, 0)).int().numpy().astype(np.uint8))
plt.imsave("True1.png", torch.squeeze(img2).cpu().permute((1,  2, 0)).int().numpy().astype(np.uint8))


code1 = autoencoder_model.module.get_code(img1)

pred1 = autoencoder_model.module.generate_from_code(code1)
pred1 = torch.squeeze(pred1).cpu().permute((1,  2, 0)).int().numpy()

code2 = autoencoder_model.module.get_code(img2)
pred2 = autoencoder_model.module.generate_from_code(code2)
pred2 = torch.squeeze(pred2).cpu().permute((1,  2, 0)).int().numpy()

plt.imshow(pred1)
plt.show()
print()
print()
print()
print()

diff = (code2-code1)/16
code = code1
for i in range(16):
    code = code + diff
    pred = autoencoder_model.module.generate_from_code(code)
    pred = torch.squeeze(pred).cpu().permute((1,  2, 0)).int().numpy()
    plt.imsave(str(i)+'.png', pred.astype(np.uint8))
    plt.imshow(pred)
    plt.show()


print()
print()
print()
print()
plt.imshow(pred2)
plt.show()


In [ ]:
localization_model = nn.DataParallel(PoseRegressor()).to('cuda')
trainable_params = sum(p.numel() for p in localization_model.parameters() if p.requires_grad)
log("Trainable parameters:", trainable_params)

In [ ]:
def localization_loss(rot_true, tr_true, rot_pred, tr_pred):
    diff_rot = rot_true-rot_pred
    diff_tr = tr_true-tr_pred

    loss = (diff_rot**2).sum(-1).mean() + 100*(diff_tr**2).sum(-1).mean()
    return loss

def train(model, autoencoder, loader, optimizer, scheduler, log_vals=[]):
    for batch, blob in enumerate(loader):
        img1, rot_true, tr_true = blob
        
        img1 = torch.squeeze(img1).to('cuda')
        rot_true = rot_true.to('cuda')
        tr_true = tr_true.to('cuda')

        optimizer.zero_grad()
        code = autoencoder.module.get_code(img1)
        rot_pred, tr_pred = model(code)
        
        loss = localization_loss(rot_true, tr_true, rot_pred, tr_pred)
        
        log_vals.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()
        

        current = batch
        print("", end='\r')
        print("Iteration: %d / %d \t|\t Loss: %5f" % (current, len(loader), loss.item()), end='\n')



localization_model.train()
autoencoder_model.eval()
epochs = 2
log_vals = []
optimizer = optim.AdamW(autoencoder_model.parameters(), lr=1e-5, weight_decay=0.001)
#optimizer = optim.Adam(model.parameters(), lr=4*1e-3)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs*len(loader), eta_min=1e-5)

autoencoder_model.train()
print("============== ", "Training", " ==============\n")
for epoch in range(epochs):
    print("---------- ", "Epoch ", epoch, " ----------\n")
    train(localization_model, autoencoder_model, loader, optimizer, scheduler, log_vals)
    plt.plot(log_vals)
    plt.show()

In [ ]:
torch.save(localization_model.state_dict(), 'LocalizationNet.pth')